# Run experiments with TraCi

In the previous tutorials, we learned how to run simulations with custom networks, and custom parameters. However, when a simulation is launched, it's not possible to modify the settings __during__ the execution. To overcome this problem, SUMO integrates a library named TraCi, allowing you to control simulations, simulation step by simulation step. At each step, it is therefore possible to modify the behavior of the infrastructures, for example, by adapting them to the traffic.

In this tutorial, we'll learn how to use TraCi with sumo-experiments.

## TraCi with pre-set networks

The pre-set networks from sumo-experiments implement functions that use TraCi. This is the main reason why detectors are also implemented. There are 3 different "TraCi functions", also called strategies in the library, that adapt traffic lights behaviour to detectors signals :
- The `boolean_detection` must be used with detectors generated by the `generate_boolean_detectors` method. When a traffic light detects a car on a lane where traffic light is red, and if it doesn't detect any car on the green lanes, it switches to the other phase if the current phase is set since more than the minimum duration time. If this condition doesn't occur, the traffic light switch to the other phase when the current phase last for more than the maximum duration time. This strategy needs three parameters : `min_duration_tl` is the minimum duration (in seconds) for a traffic light phase, `max_duration_tl` is the maximum duration (in seconds) for a traffic light phase, and `simulation_duration` is the total simulation duration.

- The `numerical_detection_all_vehicles` must be used with detectors generated by the `generate_numerical_detectors` method. When a traffic light detect more than the threshold of cars on a lane where traffic light is red, it switches to the other phase if the current phase is set since more than the minimum duration time. Both stopped and running cars are considered in this strategy. If this condition doesn't occur, the traffic light switch to the other phase when the current phase last for more than the maximum duration time. This strategy needs four parameters : `min_duration_tl` is the minimum duration for a traffic light phase, `max_duration_tl` is the maximum duration for a traffic light phase, `vehicle_threshold` is the number of vehicles detected required to switch to another phase and `simulation_duration` is the total simulation duration.

- The `numerical_detections_stopped_vehicles` must be used with detectors generated by the `generate_numerical_detectors` method. When a traffic light detect more than the threshold of cars on a lane where traffic light is red, it switches to the other phase if the current phase is set since more than the minimum duration time. Only stopped cars are considered in this strategy. If this condition doesn't occur, the traffic light switch to the other phase when the current phase last for more than the maximum duration time. This strategy needs four parameters : `min_duration_tl` is the minimum duration for a traffic light phase, `max_duration_tl` is the maximum duration for a traffic light phase, `vehicle_threshold` is the number of vehicles detected required to switch to another phase and `simulation_duration` is the total simulation duration.

To run a simulation with this strategies, you need to use the `run_traci` method from Experiment (and not the `run` method). The strategy passed as parameter will be used to control the infrastructures. Let's see a short example.

In [ ]:
from sumo_experiments.preset_networks import SquareNetwork
from sumo_experiments import Experiment

network = SquareNetwork()

exp = Experiment(
    name = 'traci',
    infrastructures = network.generate_infrastructures,
    flows = network.generate_flows_all_directions,
    detectors = network.generate_boolean_detectors
)

exp.set_parameter('square_side_length', 3)
exp.set_parameter('min_duration_tl', 15)
exp.set_parameter('max_duration_tl', 60)
exp.set_parameter('simulation_duration', 1000)
exp.set_parameter('flow_frequency', 25)

exp.run_traci(network.boolean_detection, gui=True)

exp.clean_files()

## Custom strategies

You can create your own strategies for any SUMO network. We won't describe in this tutorial how to use the TraCi library from python, we highly recommand you to check the [documentation](https://sumo.dlr.de/docs/TraCI/Interfacing_TraCI_from_Python.html).

When your strategy is defined, you can run it with the `run_traci` method from Experiment, with the correspondant network.